## Iterative (Looping) Workflows in LangGraph


---

## 1. Concept Overview

Iterative workflows differ from linear or parallel workflows by allowing **repeated execution of nodes** until a quality condition is satisfied or a hard stop is reached.
In LangGraph, iteration is implemented using **conditional edges** combined with **loop-back edges**.

Core properties:

* Deterministic control flow
* Explicit termination conditions
* State-driven evaluation
* No implicit retries

Primary goal: **progressive quality improvement through evaluation-feedback loops**.

---

## 2. Use Case: Automated Tweet Generation

Target platform: Twitter/X
Problem addressed: LLM-generated tweets are often generic or low-impact.

Solution: A **three-node iterative loop**:

1. Generate → create candidate tweet
2. Evaluate → strict quality judgment
3. Optimize → rewrite based on feedback

The loop continues until:

* The tweet is approved, **or**
* `max_iteration` is reached

---

## 3. High-Level Architecture

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/1%2AWK2y_TTKVa0bxbC_A11tWw.png)

![Image](https://images.squarespace-cdn.com/content/v1/6508aa0576c883057e96d23c/415ab2a4-782d-458c-b45f-000ed0c34f62/16%2BSystems%2BDesign%2C%2Bor%2BHow%2Bto%2BMake%2BLLMs%2BPart%2Bof%2Ba%2BFeedback%2BLoop%2B%282%29.png)

![Image](https://www.researchgate.net/profile/Rania-Khalaf/publication/37685567/figure/fig2/AS%3A394256666644492%401471009522682/State-transition-Diagram-Loops-Description-of-events.png)

Components:

* **StateGraph**: Controls execution
* **Typed State**: Shared memory across nodes
* **Reducer functions**: Preserve historical data
* **Conditional routing**: Governs looping behavior

---

## 4. State Definition

The state is the **single source of truth** for the workflow.

Key design decisions:

* Use `TypedDict` for static guarantees
* Use `Annotated[..., operator.add]` to append history instead of overwriting
* Track iteration explicitly to prevent infinite loops

```python
from typing import TypedDict, List, Annotated, Literal
import operator

class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[List[str], operator.add]
    feedback_history: Annotated[List[str], operator.add]
```

State invariants:

* `iteration <= max_iteration`
* `tweet_history[i]` corresponds to `feedback_history[i-1]`

---

## 5. Node Responsibilities

![Image](https://dist.neo4j.com/wp-content/uploads/20240618104511/build-kg-genai-e1718732751482.png)

![Image](https://miro.medium.com/1%2ABR5ProfeqWPQTZnUB_pTuw.png)

![Image](https://substackcdn.com/image/fetch/%24s_%21cBRs%21%2Cf_auto%2Cq_auto%3Agood%2Cfl_progressive%3Asteep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc1ccc0c4-b5f2-4b02-91b3-ea5bc54cd8c0_1687x993.png)

### 5.1 Generate Node

Purpose:

* Produce the **initial tweet** from topic input

Characteristics:

* High creativity
* No evaluation logic
* Appends to `tweet_history`

```python
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generator_llm.invoke(messages).content
    return {"tweet": response, "tweet_history": [response]}
```

---

### 5.2 Evaluate Node (Structured Output)

Purpose:

* Enforce **objective quality gates**
* Return **machine-readable decisions**

Key technique:

* Pydantic schema + `with_structured_output`

```python
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
```

Evaluation guarantees:

* No free-form parsing
* Deterministic routing
* Hard auto-reject rules enforced

```python
def evaluate_tweet(state: TweetState):
    response = structured_evaluator_llm.invoke(messages)
    return {
        "evaluation": response.evaluation,
        "feedback": response.feedback,
        "feedback_history": [response.feedback]
    }
```

---

### 5.3 Optimize Node

Purpose:

* Rewrite tweet using **specific critic feedback**
* Increment iteration counter

```python
def optimize_tweet(state: TweetState):
    response = optimizer_llm.invoke(messages).content
    iteration = state["iteration"] + 1

    return {
        "tweet": response,
        "iteration": iteration,
        "tweet_history": [response]
    }
```

Constraints:

* Must stay under 280 characters
* Must not introduce Q&A format
* Must directly address feedback text

---

## 6. Routing Logic (Termination Control)

![Image](https://miro.medium.com/1%2AbwFI898xU8OA1woG1FDuZA.png)

![Image](https://corporate-assets.lucid.co/chart/91f7e59b-6c97-4f0f-a808-81ccb2e89a75.jpeg?v=1707842463172)

Routing is implemented as a **pure function** (not a node).

```python
def route_evaluation(state: TweetState):
    if state["evaluation"] == "approved" or state["iteration"] >= state["max_iteration"]:
        return "approved"
    return "needs_improvement"
```

Properties:

* Side-effect free
* Deterministic
* Prevents infinite loops

---

## 7. Graph Construction

![Image](https://substackcdn.com/image/fetch/%24s_%21jnzp%21%2Cf_auto%2Cq_auto%3Agood%2Cfl_progressive%3Asteep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc1eca5d1-0462-4833-ae12-7b645a4e3a20_1828x876.png)

![Image](https://miro.medium.com/1%2AbwFI898xU8OA1woG1FDuZA.png)

```python
from langgraph.graph import StateGraph, START, END

builder = StateGraph(TweetState)

builder.add_node("generate", generate_tweet)
builder.add_node("evaluate", evaluate_tweet)
builder.add_node("optimize", optimize_tweet)

builder.add_edge(START, "generate")
builder.add_edge("generate", "evaluate")

builder.add_conditional_edges(
    "evaluate",
    route_evaluation,
    {
        "approved": END,
        "needs_improvement": "optimize"
    }
)

builder.add_edge("optimize", "evaluate")

graph = builder.compile()
```

Graph semantics:

* `evaluate` is the decision point
* `optimize → evaluate` forms the loop
* `END` is reachable from only one node

---

## 8. Execution Example

Initial state:

```python
initial_state = {
    "topic": "srhberhb",
    "iteration": 1,
    "max_iteration": 5
}
```

Invocation:

```python
result = workflow.invoke(initial_state)
```

Observed behavior:

* One generation
* One evaluation
* Approved on first pass
* No optimization loop triggered

---

## 9. Key Implementation Insights

* Iterative quality improvement requires **explicit evaluation criteria**
* Structured outputs eliminate brittle text parsing
* Reducers enable full auditability of model behavior
* Loop bounds are mandatory in production systems
* Different models per node optimize cost vs. capability

---

## 10. Mental Model Summary

* LangGraph loops are **graph-level constructs**, not Python loops
* State evolution is additive, not mutative
* Every iteration is observable and replayable
* Quality control is enforced structurally, not heuristically





```bash
pip install -U langgraph langchain langchain-openai pydantic
```

---

## Step 1: Import Core Dependencies

Purpose:

* LangGraph for workflow orchestration
* LangChain for LLM abstraction
* Pydantic for structured evaluation
* TypedDict + reducers for state safety and history tracking

```python
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import operator
```

---

## Step 2: Initialize LLMs

Design choice:

* Same model used for simplicity
* Logical separation by role (generator, evaluator, optimizer)

```python
generator_llm = ChatOpenAI(model="gpt-4o-mini")
evaluator_llm = ChatOpenAI(model="gpt-4o-mini")
optimizer_llm = ChatOpenAI(model="gpt-4o-mini")
```

---

## Step 3: Define Structured Evaluation Output

Purpose:

* Enforce deterministic routing
* Prevent free-text parsing errors

```python
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)
```

---

## Step 4: Define Shared State (Graph Memory)

Key points:

* `Annotated[..., operator.add]` appends history instead of overwriting
* Iteration counter prevents infinite loops

```python
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
```

---

## Step 5: Generation Node

Responsibility:

* Produce the first tweet
* No evaluation logic
* Append tweet to history

```python
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generator_llm.invoke(messages).content
    return {"tweet": response, "tweet_history": [response]}
```

---

## Step 6: Evaluation Node (Critic)

Responsibility:

* Enforce quality gates
* Decide approve vs improve
* Generate actionable feedback

```python
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic."),
        HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Criteria:
1. Originality
2. Humor
3. Punchiness
4. Virality
5. Format (not Q&A, under 280 chars)

Auto-reject if:
- Q&A format
- Over 280 characters
- Traditional setup–punchline jokes
- Weak generic endings

Respond ONLY in structured format.
""")
    ]

    response = structured_evaluator_llm.invoke(messages)
    return {
        "evaluation": response.evaluation,
        "feedback": response.feedback,
        "feedback_history": [response.feedback],
    }
```

---

## Step 7: Optimization Node

Responsibility:

* Rewrite tweet using critic feedback
* Increment iteration counter

```python
def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Rules:
- Stay under 280 characters
- Avoid Q&A style
""")
    ]

    response = optimizer_llm.invoke(messages).content
    return {
        "tweet": response,
        "iteration": state["iteration"] + 1,
        "tweet_history": [response],
    }
```

---

## Step 8: Routing Logic (Loop Control)

Purpose:

* Stop when approved
* Stop when max iterations reached

```python
def route_evaluation(state: TweetState):
    if state["evaluation"] == "approved" or state["iteration"] >= state["max_iteration"]:
        return "approved"
    return "needs_improvement"
```

---

## Step 9: Build the LangGraph Workflow

![Image](https://miro.medium.com/v2/resize%3Afit%3A1200/1%2AWK2y_TTKVa0bxbC_A11tWw.png)

![Image](https://images.squarespace-cdn.com/content/v1/6508aa0576c883057e96d23c/415ab2a4-782d-458c-b45f-000ed0c34f62/16%2BSystems%2BDesign%2C%2Bor%2BHow%2Bto%2BMake%2BLLMs%2BPart%2Bof%2Ba%2BFeedback%2BLoop%2B%282%29.png)

```python
graph = StateGraph(TweetState)

graph.add_node("generate", generate_tweet)
graph.add_node("evaluate", evaluate_tweet)
graph.add_node("optimize", optimize_tweet)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")

graph.add_conditional_edges(
    "evaluate",
    route_evaluation,
    {"approved": END, "needs_improvement": "optimize"},
)

graph.add_edge("optimize", "evaluate")

workflow = graph.compile()
```

---

## Step 10: Run the Workflow

```python
initial_state = {
    "topic": "srhberhb",
    "iteration": 1,
    "max_iteration": 5,
}

result = workflow.invoke(initial_state)
```

---

## Step 11: Inspect Results

Final approved tweet:

```python
result["tweet"]
```

Full iteration history:

```python
for tweet in result["tweet_history"]:
    print(tweet)
```

---

## Mental Model (Concise)

* LangGraph loops are **graph edges**, not Python loops
* Evaluation is the decision gate
* State reducers preserve full history
* Iteration limits are mandatory for safety
* Structured output enables deterministic control flow


